### Imports

In [14]:
import os
import pickle
import numpy as np

import tensorflow as tf
from tensorflow.keras.layers import TimeDistributed, Activation, Bidirectional, ConvLSTM2D, Attention, Dense, Flatten, MaxPool3D, MaxPool2D,BatchNormalization, Conv3D, GRU
from tensorflow.keras.utils import Sequence
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras import Model
from tensorflow.keras.backend import ctc_batch_cost, ctc_decode, ctc_label_dense_to_sparse

import Levenshtein as Lev
import sys

In [2]:
print(tf.__version__)

2.0.0


To Decide
1. Batch Norm vs Layer Norm or No Norm for attention model
2. Shuffle or No

To Do 
1. Check ss, sr
2. check if shapes match
3. append 0s

### Paths

In [3]:
train_path = "./LibriSpeech100/train/train_all/"
dev_path = "./LibriSpeech100/dev/dev_all/"
test_path = "./LibriSpeech100/test/test_all/"

In [4]:
class DataGenerator(Sequence):
    def __init__(self, path, to_fit = True):
        self.path = path
        self.list_X, self.list_Y = self.getLists()
        self.to_fit = to_fit
    
    def __len__(self):
        return len(self.list_X)
    
    def __getitem__(self, index):      
        dict_X = self.get_dict_X(index)   
        
        if self.to_fit:
            dict_Y = self.get_dict_Y(index)
            X, Y = self.generate_XY(dict_X, dict_Y)
            return X, Y
        
        X = self.generate_X(dict_X)
        return X
    
    def getLists(self):
        list_X = []
        list_Y = []
        for item in sorted(os.listdir(self.path)):
            ext = item.split(".")[-1]
            if ext == 'pkl':
                list_X.append(item)
            elif ext == 'txt':
                list_Y.append(item)
        return list_X, list_Y
    
    def get_dict_X(self, index):
        file_name = self.path + self.list_X[index]
        with open(file_name, 'rb') as pickle_file:
            dict_X = pickle.load(pickle_file)
        return dict_X
    
    def get_dict_Y(self, index):
        file = open(train_path+filename)
        dict_Y = {}
        for line in file:
            data = line.split()
            key = data[0]
            value = ' '.join(data[1:])
            dict_Y[key] = value
        return dict_Y
    
    def generate_XY(self, dict_X, dict_Y):
        X = []
        Y = []
        for key in dict_X:
            X.append(dict_X[key])
            Y.append(dict_Y[key])
        return X, Y
    
    def generate_X(self, dict_X):
        X = [value.T for key, value in dict_X.items()]
        X = tf.keras.preprocessing.sequence.pad_sequences(X, padding = 'post')
        return X

In [5]:
def wer(s1, s2):

    s1 =s1.lower()
    s2 =s2.lower()
    b = set(s1.lower().split() + s2.lower().split())
    
    word2char = dict(zip(b, range(len(b))))


    w1 = [chr(word2char[w]) for w in s1.split()]
    w2 = [chr(word2char[w]) for w in s2.split()]
    return Lev.distance(''.join(w1), ''.join(w2))/float(len(s2.split()))

In [7]:
train_data = DataGenerator(train_path)
val_data = DataGenerator(dev_path)
test_data = DataGenerator(test_path)

In [13]:
class BaseModel(Model):
    def __init__(self, op_dim = 28):
        super(BaseModel, self).__init__()
        self.rnn = GRU(output_dim, return_sequences= False)
        self.y_pred = Activation('softmax')
        
    def call(self, inputs):
        x = self.rnn(inputs)
        x = self.y_pred(x)
        return x

In [ ]:
class ASRModel(Model):
    def __init__(self):
        super(ASRModel, self).__init__()
        self.base_model = BaseModel()
        self.activation = Activation('softmax')

        
    def call(self, inputs):
        input_data, labels, input_length, label_length = inputs
        x = self.base_model(input_data)
        x = self.activation()          
        x = K.ctc_batch_cost(labels, x, input_length, label_length)
        return x
        

In [ ]:
model = ASRModel(inputs = [input_data, labels, input_length, label_length])
model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer=tf.keras.optimizers.Adam(), metrics=[wer])